In [3]:
import os
vipshome = r'C:\Users\labadmin\Documents\vips-dev-8.14\bin'
os.environ['PATH'] = vipshome + ';' + os.environ['PATH']
import pyvips
import numpy as np
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import time
from tqdm import tqdm

In [4]:
def return_row_col(image_dir):
    imlist = [x for x in os.listdir(image_dir) if x.endswith('.png')]
    min_x = float('inf')
    min_y = float('inf')
    max_x = float('-inf')
    max_y = float('-inf')
    start = time.time()
    for filename in tqdm(imlist, desc="Number of images processed", colour='red'):
        x, y = map(int, filename.split('xy')[0].split('.png')[0].split('_'))
        if x < min_x:
            min_x = x
        if y < min_y:
            min_y = y
        if x > max_x:
            max_x = x
        if y > max_y:
            max_y = y
    end = time.time()
    max_min_ra = min_x, min_y, max_x, max_y
    num_row = int((max_x + 1024 - min_x) / 1024)
    num_col = int((max_y + 1024 - min_y) / 1024)
    print("time it took: {} minutes".format((round(end - start) / 60), 3))
    return max_min_ra, num_row, num_col

def generate_5d_array(image_dir, min_x, min_y, num_rows, num_cols):
    array_5d = np.zeros((num_cols, num_rows, 1024, 1024, 3), dtype=np.uint8)
    imlist = [x for x in os.listdir(image_dir) if x.endswith('.png')]
    start = time.time()
    for filename in tqdm(imlist, desc="Number of images processed", colour='red'):
        x, y = map(int, filename.split('xy')[0].split('.png')[0].split('_'))
        x = int((x - min_x) / 1024)
        y = int((y - min_y) / 1024)
        image = Image.open(os.path.join(image_dir, filename))
        image = np.array(image)
        array_5d[y, x, :, :, :] = image
    end = time.time()
    print("time it took: {} minutes".format((round(end - start) / 60), 3))
    return array_5d


In [5]:
tile_src = r'\\shelter\Kyu\unstain2stain\tiles\registrated_tiles\Syn_HE\OTS_14684_1_parfor'
max_min_ra, num_row, num_col = return_row_col(tile_src)
min_x = max_min_ra[0]
min_y = max_min_ra[1]
print("min_x is {}, min_y is {}, num_row is {}, num_col is {}".format(min_x, min_y, num_row, num_col))

#slow to open image in for 12min
tiles = generate_5d_array(image_dir = tile_src,min_x = min_x,min_y = min_y,num_rows = num_row,num_cols = num_col)
print("shape of tiles is {}".format(tiles.shape))

stitched_wsi = np.reshape(tiles.swapaxes(1,2),(1024*num_col,1024*num_row,3))
print("shape of reconstructed wsi is {}".format(stitched_wsi.shape))

Number of images processed: 100%|██████████| 12905/12905 [00:00<00:00, 1433878.86it/s]


time it took: 0.0 minutes
min_x is 8801, min_y is 787, num_row is 145, num_col is 89


Number of images processed: 100%|██████████| 12905/12905 [11:45<00:00, 18.30it/s]


time it took: 11.75 minutes
shape of tiles is (89, 145, 1024, 1024, 3)
shape of reconstructed wsi is (91136, 148480, 3)


In [6]:
start = time.time()
im = pyvips.Image.new_from_array(stitched_wsi)
image_height = im.height
image_bands = im.bands
print("imheight imwidth", image_height, im.width)
im = im.copy()
im.set_type(pyvips.GValue.gint_type, "page-height", image_height)
im.set_type(pyvips.GValue.gstr_type, "image-description",
            f"""<?xml version="1.0" encoding="UTF-8"?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
    <Image ID="Image:0">
        <!-- Minimum required fields about image dimensions -->
        <Pixels DimensionOrder="XYCZT"
                ID="Pixels:0"
                SizeC="{image_bands}"
                SizeT="1"
                SizeX="{im.width}"
                SizeY="{image_height}"
                SizeZ="1"
                Type="uint8">
        </Pixels>
    </Image>
</OME>""")
end = time.time()
print("time it took to set ometiff up took {} minutes".format((round(end - start) / 60), 3))

#12min
start = time.time()
dst = os.path.join(tile_src,'stitched')
if not os.path.exists(dst): os.mkdir(dst)
im.tiffsave(os.path.join(dst, 'stitched.ome.tiff'), compression="jpeg", tile=True, tile_width=512,
            tile_height=512, pyramid=True, subifd=True)
end = time.time()
print("time it took to save ometiff took {} minutes".format((round(end-start)/60),3))


imheight imwidth 91136 148480
time it took to set ometiff up took 0.0 minutes
time it took to save ometiff took 12.15 minutes
